In [1]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
def get_page_info(url):
    
    r = req.get(url)
    soup = BeautifulSoup(r.text)
    
    time = []
    types = []
    address = []
    deal_price = []
    floor_space = []
    unit_price = []
    plot = []
    floor = []
    age = []
    parkingspace = []
    info = []

    for i in soup.find_all("td", class_="time"):
        time.append(i.text.strip().replace(" ", "").replace("\r", "").replace("\n", ""))
    info.append(time)

    for i in soup.find_all("td", class_="type"):
        types.append(i.text.strip().replace(" ", "").replace("\r", "").replace("\n", ""))
    info.append(types)    

    for i in soup.find_all("td", class_="add"):
        address.append(i.text.strip().replace(" ", "").replace("\r", "").replace("\n", ""))
    info.append(address) 

    for i in soup.find_all("td", class_="dealPrice"):
        deal_price.append(i.text.strip().replace(" ", "").replace("\r", "").replace("\r",""))
    info.append(deal_price) 

    for i in soup.find_all("td", class_="floorSpace"):
        floor_space.append(i.text.strip().replace(" ", "").replace("\r", "").replace("\r",""))
    info.append(floor_space) 

    for i in soup.find_all("td", class_="unitPrice"):
        unit_price.append(i.text.strip().replace(" ", "").replace("\r", "").replace("\n", ""))
    info.append(unit_price) 

    for i in soup.find_all("td", class_="plot"):
        plot.append(i.text.strip().replace(" ", "").replace("\r", "").replace("\n", ""))
    info.append(plot) 

    for i in soup.find_all("td", class_="floor"):
        floor.append(i.text.strip())
    info.append(floor) 

    for i in soup.find_all("td", class_="age"):
        age.append(i.text.strip().replace(" ", "").replace("\r", "").replace("\n", ""))
    info.append(age) 

    for i in soup.find_all("td", class_="parkingSpace"):
        parkingspace.append(i.text.strip().replace(" ", "").replace("\r", "").replace("\n", ""))
    info.append(parkingspace) 
    
    title = ['成交', '型態', '地址', '成交價', '建坪', '單價', '地坪', '樓別', '屋齡', '車位']
    
    data = {}
    for j in range(len(title)):
        data[title[j]] = info[j]

    data = pd.DataFrame(data)
   
    return data

taipei = get_page_info("https://evertrust.yungching.com.tw/region/台北市/不限/1#mainContent").reset_index().drop("index", axis = 1)
taipei.head()

,成交,型態,地址,成交價,建坪,單價,地坪,樓別,屋齡,車位
0,10904,車位,中山區民權東路一段21~30號,220萬,12.21坪,--萬,0.26坪,-3~-3/15,11.3年,有
1,10904,其他,萬華區大理街31~40號,120萬,3.13坪,38.3萬,0.36坪,-1~-1/4,44.3年,無
2,10903,套房,士林區中正路104巷1弄1~10號格局：0房(室)1廳1衛,580萬,9.92坪,58.5萬,1.94坪,2~2/5,24.4年,無
3,10903,公寓,士林區承德路五段11~20號格局：3房(室)2廳2衛,"1,100萬",34.98坪,31.4萬,11.65坪,3~3/4,45.3年,無
4,10903,套房,綠光新第士林區士東路89巷11~20號格局：1房(室)1廳1衛,545萬,8.96坪,60.8萬,2.90坪,2~2/6,25.6年,無


In [ ]:
def get_web_url(url):
    
    r1 = req.get(url)
    soup1 = BeautifulSoup(r1.text)
    for i in soup1.find("ul", class_="pagination clearfix"):
        
        if i.find("a") != -1 and "下一頁" in i.find("a").text:
            notskip = True
        else:
            notskip = False
            
        while notskip:
            next_page = "https://evertrust.yungching.com.tw"+ i.find("a").get("href")
            url = next_page             
            return url

In [4]:
url = "https://evertrust.yungching.com.tw/region/台北市/不限/1#mainContent"
search_cities = []

r = req.get(url)
soup = BeautifulSoup(r.text)

for i in soup.find("ul", class_="conditions"):
    if i.find("a") != -1:
        city = i.find("a").get("title").strip()
        city_name = i.find("a").text.strip()

        search_cities.append(city_name)
print(search_cities)

['北市', '新北', '桃園', '竹市', '竹縣', '台中', '彰化', '台南', '高雄', '屏東', '宜蘭', '基隆', '南投', '雲林', '苗栗', '嘉市', '嘉縣', '花蓮', '台東', '金門', '澎湖']


In [ ]:
import sys

for city in search_cities:
    first_page = "https://evertrust.yungching.com.tw/region/{}/不限/1#mainContent".format(city)

    current_page = first_page

    #next_page = get_web_url(current_page)
    #dataframe = get_page_info(current_page)

    dataframe= []
    while current_page:
    
        data = get_page_info(current_page)
        dataframe.append(data)
        current_page = get_web_url(current_page)
    
    sys.stderr.write("Write data to file "+city+"...\n")
    full_data = pd.concat(dataframe)

    writer = pd.ExcelWriter("C:/Users/b5031/housing.xlsx")
    full_data.to_excel(writer, encoding = "big5", sheet_name = city)
    writer.save()